### Structure of DataFrames
* **Provences**: [Provence, Domain, Regent, Terrain, Loyalty, Taxation, Population, Magic, Castle, Capital, Position, Troops]
* **Holdings**: [Provence, Domain, Regent, Type, Level]
* **Regents**: [Regent, Full Name, Player, Class, Level, Alignment, Str, Dex, Con, Int, Wis, Cha, Insight, Deception, Persuasion, Regency Points, Gold Bars, Regency Bonus, Attitude, Lieutenants]
* **Geography**: [Provence, Neighbor, Border, Road, Caravan, Shipping]
* **Relationship**: [Regent, Other, Diplomacy, Payment, Vassalage]
* **Troops**: [Regent, Provence, Type, Cost, CR]

* **Seasons**: A dctionary of season-dataframes (to keep track of waht happened)



In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output

Game = Regency()
clear_output()
for a in range(1):
    Game.random_events(Threshold=0)
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_regency_points()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.domain_initiative()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_gold_bars()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.maintenance_costs()
    print(len((Game.Seasons[Game.Season]['Season'])))


Game.Seasons[Game.Season]['Season']

22
22
22
Instructions for updating:
Use tf.cast instead.


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


22
22


,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,TBM,False,No Event,19,20,0,0,0,Dormant
1,Ch,False,No Event,28,19,0,0,0,Dormant
2,Sw2,False,No Event,24,19,1,0,2,Bare
3,LPA,False,No Event,11,18,7,11,4,Bare
4,MF,False,No Event,59,17,24,5,9,Rich
5,IHH,False,No Event,6,16,0,3,3,Bare
6,Is,False,No Event,106,15,5,15,10,Average
7,DW,False,No Event,70,15,34,12,11,Rich
8,Sw,False,No Event,11,13,0,0,0,Dormant
9,JR,False,No Event,33,13,-1,5,6,Dormant


In [ ]:
Game.Regents

In [ ]:
import pandas as pd

Game.random_events(Threshold=.999)
df = Game.Seasons[Game.Season]['Season']
self = Game
df

In [ ]:
import numpy as np
df = Game.Seasons[Game.Season]['Season']
self = Game
# start

df = pd.merge(df, self.Regents[['Regent', 'Gold Bars']], on='Regent', how='left')
df['Cost'] = np.random.randint(1,6,df.shape[0])

# not allowed to geo negative
df1 = df[df['Cost']<df['Gold Bars']]
df2 = df[df['Cost']>=df['Gold Bars']]
df2['Cost'] = df2['Gold Bars']
df = pd.concat([df1, df2])

temp = pd.merge(df['Regent'], self.Provences[['Provence', 'Regent', 'Loyalty']].copy(), on='Regent', how='left').fillna('Rebellious')
temp = temp[temp['Loyalty']!='Rebellious'].copy()  # pointless
temp['Loyalty'] = temp['Loyalty'].str.replace('Poor','1').replace('Average','2').replace('High',3).astype(int)
temp = pd.merge(temp[['Regent', 'Loyalty']].groupby('Regent').min().reset_index(),temp,on=['Regent', 'Loyalty'], how='left')


df = pd.merge(temp, df, on='Regent', how='left')
df = df[df['Provence']!='High'].copy()


df['Loyalty'] = df['Loyalty'].astype(str).str.replace('1', 'Poor').replace('2', 'Poor').replace('3','Average')
lst = []
for i, row in df.iterrows():
    if row['Regent'] not in lst:
        self.change_regent(row['Regent'], Gold_Bars = row['Gold Bars'] - row['Cost'])
        self.change_provence(row['Provence'], Loyalty=row['Loyalty'])
        lst.append(row['Regent'])
df

In [ ]:
self.Regents

In [ ]:
Game.Provences

In [ ]:
self = Game

In [ ]:
self.Relationships[['Other', 'Vassalage']].copy()